# Travailler avec le calcul

Quand vous exécutez un script en tant que travail Azure Machine Learning, vous devez définir son contexte d’exécution. Une configuration clé est la cible de calcul sur laquelle le script sera exécuté. Il peut s’agir de la station de travail locale (dans ce cas, l’instance de calcul) ou d’une cible de calcul distante telle que le cluster de calcul managé Azure Machine Learning qui est provisionné à la demande.

Dans ce notebook, vous allez créer un cluster de calcul et explorer les cibles de calcul pour les travaux.

## Avant de commencer

Vous devez avoir la dernière version du package **azureml-ai-ml** pour exécuter le code dans ce notebook. Exécutez la cellule ci-dessous pour vérifier qu’il est installé.

> **Remarque** :
> Si le package **azure-ai-ml** n’est pas installé, exécutez `pip install azure-ai-ml` pour l’installer.

In [ ]:
## Se connecter à un espace de travail

Une fois les packages requis du Kit de développement logiciel (SDK) installés, vous êtes prêt à vous connecter à votre espace de travail.

Pour vous connecter à un espace de travail, vous avez besoin de paramètres d’identificateur : un ID d’abonnement, un nom de groupe de ressources et un nom d’espace de travail. Le nom du groupe de ressources et le nom de l’espace de travail sont déjà renseignés pour vous. Vous avez seulement besoin de l’ID d’abonnement pour exécuter la commande.

Pour trouver les paramètres nécessaires, cliquez sur l’abonnement et le nom de l’espace de travail en haut à droite du studio. Un volet s’ouvre à droite.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copiez l’ID d’abonnement et remplacez **YOUR-SUBSCRIPTION-ID** par la valeur que vous avez copiée. </p>

## Créer un cluster de calcul

Dans de nombreux cas, il se peut que vos ressources de calcul locales ne soient pas suffisantes pour traiter un essai complexe ou de longue durée qui doit traiter un volume important de données, et vous pouvez tirer parti de la possibilité de créer et d’utiliser de façon dynamique des ressources de calcul dans le cloud. Azure Machine Learning prend en charge une plage de cibles de calcul, que vous pouvez définir dans votre espace de travail et utiliser pour exécuter des travaux, en payant les ressources uniquement quand vous les utilisez.

Vous pouvez créer un cluster de calcul dans [Azure Machine Learning Studio](https://ml.azure.com) en utilisant le SDK Python ou Azure CLI. La cellule de code suivante recherche dans votre espace de travail l’existence d’un cluster de calcul nommé `aml-cluster`. S’il n’existe pas, créez-le.

In [ ]:
Une fois que vous avez créé un cluster de calcul, vous pouvez uniquement modifier la configuration pour :

- `min_instances` : nombre minimal de nœuds.
- `max_instances` : nombre maximal de nœuds.
- `idle_time_before_scale_down` : temps d’inactivité avant scale-down

Actuellement, votre cluster de calcul `aml-cluster` ne peut effectuer le scale-down que d’un nœud maximum. Remplaçons cela par deux, pour autoriser le calcul parallèle.

In [ ]:
Lorsque le cluster de calcul est mis à jour, vous pouvez vérifier sa configuration en imprimant ses attributs.

## Créer un script pour entraîner un modèle

Pour entraîner un modèle, vous allez d’abord créer le script **diabetes_training.py** dans le dossier **src**. Le script utilise le fichier **diabetes.csv** dans le même dossier que les données d’entraînement.

In [ ]:
## Exécuter un travail sur un cluster de calcul

Vous êtes maintenant prêt à exécuter le travail sur le cluster de calcul que vous avez créé.

> **Remarque** :
> Le travail met un certain temps à démarrer, car le cluster de calcul doit passer de zéro à un nœud. Une fois que le cluster de calcul est prêt, le script s’exécute. Une fois que le travail est terminé, le cluster de calcul redescend à zéro nœud. Vous pouvez consulter l’état du cluster de calcul dans la page **Calcul**.

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)